In [1]:
import os, sys
import numpy as np
import matplotlib.pyplot as plt

In [2]:
sys.path.append('../readmaroc')

In [3]:
%matplotlib qt

In [4]:
from maroc_data import MarocData

In [5]:
from scipy.stats import norm

In [6]:
from functools import partial

In [7]:
import matplotlib.backends.backend_pdf

In [8]:
input_dat = '../data/Run000133072.dat'

In [9]:
marocdata = MarocData(input_dat)

In [10]:
y_offset = [12000, 10000, 8000, 4000, 2000]

In [14]:
def overthreshold(marocdata, event_id):
    events = marocdata.get_event(event_id)
    count = 0
    for board_id, event in events:
        
        if event is None:
            continue
        board = marocdata.get_board(board_id)
        pedestal = board.avg_data
        noise = board.noise
        mu, std = norm.fit(noise)
        signal = event.signal
        if np.any(signal-pedestal>=100+(mu+5*std)):
            count+=1
    if count>=2:
        return True
    return False

In [15]:
event_ids_over_theshold = list(filter(partial(overthreshold, marocdata), range(1, marocdata.max_evt+1)))

In [16]:
len(event_ids_over_theshold)

61

In [36]:
def plot_event(evt, marocdata):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16,10), sharey=True, sharex=True)
    for i, (triplet_y, triplet_x) in enumerate(zip(np.arange(1, 16).reshape(5, 3), 
                                                   np.arange(16,31).reshape(5,3))):
        for j, (board_y, board_x) in enumerate(zip(triplet_y, triplet_x)):
            if board_y in marocdata.active_boards:
                yboard = marocdata.get_board(board_y)
                if evt in yboard:
                    signal = yboard.get_event(evt).signal
                    ref_event = marocdata.get_board(board_y).event_ids[0]
                    
                    TS_evt = yboard.get_event(evt).TS - yboard.get_event(ref_event).TS
                    
                    ax1.text((320+320*j-0+(j*320))/2 -150, y_offset[i]-250, 'board {}, TS: {} '.format(board_y, TS_evt), size='small')
                    if np.max(signal)>2000:
                        signal = signal*0.45
                    ax1.plot(np.arange(0+(j*320), 320+320*j), 
                             (signal)+y_offset[i], color='blue', linewidth=1)
                    ax1.axvline(320*(j+1), linestyle='--', linewidth=.75, color='grey', alpha=.5)
                else:
                    pass
            if board_x in marocdata.active_boards:
                xboard = marocdata.get_board(board_x)
                if evt in xboard:
                    
                    signal = xboard.get_event(evt).signal
                    ref_event = marocdata.get_board(board_x).event_ids[0]
                    
                    TS_evt = xboard.get_event(evt).TS - xboard.get_event(ref_event).TS
                    ax2.text((320+320*j-0+(j*320))/2 -150, y_offset[i]-250, 'board {}, TS:{} '.format(board_x, TS_evt), size='small')
                    if np.max(signal)>2000:
                        signal = signal*0.45
                    ax2.plot(np.arange(0+(j*320), 320+320*j), 
                             (signal)+y_offset[i], color='red', linewidth=1)
                    ax2.axvline(320*(j+1), linestyle='--', linewidth=.75, color='grey', alpha=.5)
                else:
                    pass
            
    ax1.set_title('y layers', size='x-large')
    ax2.set_title('x layers', size='x-large')
    plt.yticks(y_offset, ['layer 0','layer 1', 'layer 2', 'layer 3', 'layer 4'], size='x-large')
    plt.xticks([0, 320, 640, 960])
    fig.text(.5, .05, 'strips', size='large')
    fig.text(.5, .95, 'Evt {}'.format(evt), size='large')
    plt.ylim(1000, 13000)
    return fig, ax1, ax2



In [40]:
for evt in range(100):
    fig, ax1, ax2 = plot_event(evt, marocdata)
    plt.show()
    if evt==20:
        break

<ipython-input-36-719431f213bb>:2: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16,10), sharey=True, sharex=True)


In [38]:
pdf = matplotlib.backends.backend_pdf.PdfPages("output_new_tboard.pdf")
for evt in event_ids_over_theshold:
    fig, ax1, ax2 = plot_event(evt, marocdata)
    pdf.savefig(fig)
    plt.close(fig)
pdf.close()
plt.close()

In [50]:
list(marocdata.get_board(8).timestamps.values()) - list(marocdata.get_board(8).timestamps.values())[0]

array([         0,    3522772,    4994537, ..., 4283839456, 4290466592,
       4291107875], dtype=uint32)

In [51]:
list(marocdata.get_board(9).timestamps.values()) - list(marocdata.get_board(9).timestamps.values())[0]

array([         0,    3522772,    4994537, ..., 4283839456, 4290466592,
       4291107875], dtype=uint32)

In [60]:
marocdata.get_board(9).event_ids

array([  23,   24,   25, ..., 1264, 1265, 1266], dtype=uint32)

In [61]:
marocdata.get_board(9).get_event(24).TS-marocdata.get_board(9).get_event(23).TS

9658433

In [54]:
list(marocdata.get_board(10).timestamps.values()) - list(marocdata.get_board(10).timestamps.values())[0]

array([         0,    1471766,    8258549, ..., 4286943821, 4287585104,
       4291444525], dtype=uint32)

In [56]:
marocdata.get_board(10).timestamps

{1073: 2524504,
 1074: 3996270,
 1075: 10783053,
 1076: 16895517,
 1077: 20453031,
 1078: 23391681,
 1079: 25394855,
 1080: 27626152,
 1081: 28228864,
 1082: 33166480,
 1083: 40762618,
 1084: 42174385,
 1085: 42403171,
 1086: 43178647,
 1087: 46548443,
 1088: 52829699,
 1089: 57796377,
 1090: 58694092,
 1091: 67590047,
 1092: 71284879,
 1093: 74372056,
 1094: 75301110,
 1095: 78330756,
 1096: 79080028,
 1097: 82140105,
 1098: 83224526,
 1099: 89213438,
 1100: 92389966,
 1101: 92400142,
 1102: 97037512,
 1103: 102632677,
 24: 103334371,
 25: 104828461,
 26: 112759838,
 1104: 113455044,
 1105: 114971720,
 1106: 114983314,
 27: 115390365,
 1107: 116091136,
 1108: 118510460,
 1109: 120459576,
 28: 122831545,
 1110: 123739187,
 1111: 124960974,
 1112: 125853971,
 1113: 129424076,
 29: 136597242,
 1114: 136712013,
 30: 143502679,
 31: 144572522,
 32: 147698643,
 33: 149983894,
 34: 157920656,
 1115: 160863193,
 35: 161000024,
 1116: 161208104,
 36: 161753664,
 37: 165358981,
 1117: 167075095

In [57]:
marocdata.get_board(10).event_ids

array([  23,   24,   25, ..., 1264, 1265, 1266], dtype=uint32)

In [59]:
marocdata.get_board(10).get_event(24).TS-marocdata.get_board(10).get_event(23).TS

9658433

In [72]:
for i in marocdata.active_boards:
    first = marocdata.get_board(i).event_ids[0]
    print(i, first, 
          marocdata.get_board(i).get_event(first).TS, 
          marocdata.get_board(i).get_event(first+1).TS, 
          marocdata.get_board(i).get_event(first+1).TS-marocdata.get_board(i).get_event(first).TS)

1 26 107782967 117441400 9658433
2 26 106215686 115874119 9658433
3 26 104663747 114322180 9658433
4 26 103122924 112781357 9658433
5 26 101529252 111187685 9658433
6 26 99978729 109637161 9658432
7 25 98399290 108057723 9658433
8 25 96776127 106434559 9658432
9 23 95306282 104964715 9658433
10 23 93675938 103334371 9658433
11 23 92171594 101830027 9658433
12 21 90559002 100217435 9658433
13 21 88972490 98630923 9658433
14 20 87403677 97062110 9658433
15 20 85797165 95455598 9658433
18 5 20017336 29675769 9658433
19 5 18416901 28075334 9658433
20 5 16900251 26558684 9658433
21 5 15334550 24992983 9658433
22 3 13757400 23415833 9658433
23 3 12113748 21772181 9658433
24 3 10539974 20198407 9658433
25 3 9002307 18660740 9658433
26 2 7438991 17097424 9658433
27 1 5876827 15535260 9658433
28 1 4329255 13987688 9658433
29 1 2714480 12372913 9658433
30 1 1161702 10820135 9658433


In [67]:
marocdata.get_board(28).get_event(1).TS

4329255

In [69]:
marocdata.get_board(26).get_event(2).TS

7438991

In [76]:
list(marocdata.get_board(8).timestamps.values())

[2101921,
 5624693,
 7096458,
 13883242,
 19995705,
 23553220,
 26491869,
 28495043,
 30726341,
 31329052,
 36266669,
 43862807,
 45274573,
 45503359,
 46278835,
 49648631,
 55929887,
 60896565,
 61794280,
 70690235,
 74385067,
 77472245,
 78401299,
 81430945,
 82180216,
 85240294,
 86324715,
 92313627,
 95490154,
 95500330,
 100137701,
 105732866,
 106434559,
 107928649,
 115860026,
 116555233,
 118071909,
 118083503,
 118490553,
 119191325,
 121610648,
 123559764,
 125931734,
 126839376,
 128061162,
 128954160,
 132524264,
 139697430,
 139812201,
 146602867,
 147672710,
 150798832,
 153084082,
 161020844,
 163963382,
 164100212,
 164308293,
 164853852,
 168459169,
 170175283,
 170302383,
 171615019,
 171805013,
 177734509,
 180549357,
 185710202,
 189488831,
 196017176,
 198185854,
 200076623,
 201084655,
 203693156,
 205735437,
 214291670,
 215222148,
 215825907,
 216513373,
 217899683,
 224685356,
 225420603,
 226173033,
 229336793,
 229405982,
 229819098,
 229990092,
 230329545,
 